<a href="https://colab.research.google.com/github/drdrwenski/sympy-examples/blob/main/MetricTensorEuclideanSpace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metric Tensor in Euclidean Space

The metric tensor is a fundamental concept in the field of differential geometry and general relativity. It is a mathematical object that encodes the intrinsic geometry of a space, allowing us to measure distances, angles, and volumes within that space.
At its core, the metric tensor is a second-rank tensor that describes the local curvature and distance relationships in a given coordinate system. It is a symmetric, positive-definite matrix that transforms covariantly under changes of the coordinate system. This means that the metric tensor contains all the information needed to perform measurements and calculations within the space, regardless of how the coordinates are defined.

##Preliminaries

First, upgrade to the latest version of SymPy if necessary. This should be at least **v1.10**.

In [1]:
%%capture
%pip install --upgrade sympy

Next, we define some output helper functions, providing shorthand notations for derivatives in curvilinear coordinates (still not working perfectly well).

In [2]:
#@title Output Helper Functions
from IPython.display import display, Markdown
from sympy import latex

def displayLatex(expr, **kwargs):
  intro = kwargs.get('intro', '')
  output = rf'$\displaystyle {intro} {latex(expr)}$'
  display(Markdown(output))

def displayLeibniz(expr,func,sys):
  leibnizComplete = ''
  for term in expr.args:
    leibniz = latex(term)
    # find second derivatives
    for i,j in zip(range(sys.dim),range(sys.dim)):
      coordStr0 = latex(sys.base_scalar(i))
      coordStr1 = latex(sys.base_scalar(j))
      # original notation
      replStr = latex(Differential(Differential(func)(sys.base_vector(i)))(sys.base_vector(j)))
      # shorthand notation
      if i == j:
        subsStr = rf'\frac{{\partial^{{2}} {func.name}}}{{\partial {coordStr0}^{{2}}}}'
      else:
        subsStr = rf'\frac{{\partial^{{2}} {func.name}}}{{\partial {coordStr1} \partial {coordStr0}}}'
      if leibniz.find(replStr) >= 0:
        leibniz = leibniz.replace(replStr,'')
        leibniz = leibniz + subsStr
    # find first derivatives
    for i in range(sys.dim):
      coordStr = latex(sys.base_scalar(i))
      baseStr = latex(sys.base_vector(i))
      # original notation
      replStr = latex(Differential(func)(sys.base_vector(i)))
      # shorthand notation
      subsStr = rf'\frac{{\partial {func.name}}}{{\partial {coordStr}}}'
      if leibniz.find(replStr) >= 0:
        leibniz = leibniz.replace(replStr,'')
        leibniz = leibniz + subsStr
      # rearrange terms
      if leibniz.find(baseStr) >= 0:
        leibniz = leibniz.replace(baseStr,'')
        leibniz = leibniz + baseStr
    # rebuild sum of derivatives
    if len(leibnizComplete) == 0:
      leibnizComplete = leibniz
    else:
      leibnizComplete = leibnizComplete + '+' + leibniz
  leibnizComplete = leibnizComplete.replace(rf'\frac{{ }}',rf'\frac{{1}}')
  leibnizComplete = leibnizComplete.replace(rf'\frac{{}}',rf'\frac{{1}}')
  display(Markdown(rf'$\displaystyle {leibnizComplete}$'))

##Curvilinear Coordinates

Spherical & cylindrical 3D coordinates are introduced utilizing the vector module. The Cartesian coordinate system serves as the parent instance for all other types of coordinate systems since we are considering Euclidean space. Coordinate transformations are then stored as symbolic expressions in a Python dictionary *relations*.
Additional dictionaries provide simplification methods tailored to each coordinate system, enhancing computational efficiency and readability of results.

In [11]:
from sympy.vector import CoordSys3D
Cart = CoordSys3D('Cart')
Spher = Cart.create_new('Spher', transformation='spherical')
Cyl = Cart.create_new('Cyl', transformation='cylindrical')

# Since there is no built-in transformation_to_parent_function() yet,
# we apply the following workaround.
def transformation_to_parent_function(child, scalars):
  sc = (scalars, child.base_scalars())
  abbrev = [(sc[1][i],sc[0][i]) for i in range(3)]
  return tuple(expr.subs(abbrev) for expr in child.transformation_to_parent())

from sympy import symbols, Function, Lambda
from sympy import sqrt
x, y, z = symbols('x y z', real=True)
r, theta, phi = symbols('r theta phi', nonnegative=True)

# Python dictionary of coordinate transformations
relations = {
        ('Cartesian','Spherical'):
            Lambda((x,y,z), Spher.transformation_from_parent_function()(x,y,z)),
        ('Spherical','Cartesian'):
            Lambda((r,theta,phi), transformation_to_parent_function(Spher, (r,theta,phi))),
        ('Cartesian','Cylindrical'):
            Lambda((x,y,z), Cyl.transformation_from_parent_function()(x,y,z)),
        ('Cylindrical','Cartesian'):
            Lambda((r,theta,z), transformation_to_parent_function(Cyl, (r,theta,z)))
}

# Python dictionaries of simplification strategies
from sympy import simplify, expand, factor, cancel
from sympy import expand_power_base, powdenest
from sympy.simplify.fu import fu, TR6
from functools import reduce
from operator import mul

simplifications = {
    'Spherical': (lambda elem: expand(TR6(factor(elem)))),
    'Cylindrical': (lambda elem: expand(TR6(factor(elem))))
}

powers = {
    'Spherical': (lambda elem: powdenest(expand_power_base(elem, force=True), force=True)),
    'Cylindrical': (lambda elem: powdenest(elem, force=True))
}

We also provide a coordinate transformation from an ellipsoidal coordinate system to a Cartesian coordinate system.

In [17]:
# Check assumptions:
xi, eta, zeta = symbols('xi eta zeta', real=True)
a, b, c = symbols('a b c', nonnegative=True)

exprs = tuple(
    sqrt(reduce(mul, [(coord**2+ellips) for ellips in (xi,eta,zeta)])/
         reduce(mul, [(cart**2-coord**2) for cart in (a,b,c) if cart != coord]))
    for coord in (a,b,c)
)

relations[('Ellipsoidal','Cartesian')] = Lambda((xi,eta,zeta), exprs)
simplifications['Ellipsoidal'] = (lambda elem: factor(cancel(elem)))
powers['Ellipsoidal'] = (lambda elem: elem)

displayLatex(exprs[0])

$\displaystyle  \sqrt{\frac{\left(a^{2} + \eta\right) \left(a^{2} + \xi\right) \left(a^{2} + \zeta\right)}{\left(- a^{2} + b^{2}\right) \left(- a^{2} + c^{2}\right)}}$

##Metric Tensor

We set up a 3-dimensional manifold, define a patch on that manifold, and create two coordinate systems on that patch: one Cartesian and one of a user-chosen type ('Spherical', 'Cylindrical' or 'Ellipsoidal').

In [19]:
from sympy.diffgeom import Manifold, Patch, CoordSystem, Point
M = Manifold('M', 3)
P = Patch('P', M)

# Try any of 'Spherical', 'Cylindrical', 'Ellipsoidal'
CoordSystemType = 'Spherical'
CoordSystemSymbols = relations.get((CoordSystemType,'Cartesian')).args[0]

R3 = CoordSystem('Cartesian', P, (x,y,z), relations)
S0 = CoordSystem(CoordSystemType, P, CoordSystemSymbols, relations)

# point = Point(S0, (1,0,0))
# displayLatex(point.coords(R3))

In applications the metric tensor with respect to curvilinear coordinates is usually displayed either as a quadratic form $\ ds^2$ or as a matrix $\ (g_{ij})$.

In [20]:
from sympy.matrices import Matrix, diag
from sympy.diffgeom import TensorProduct

nrange = range(S0.dim)

Jacob = S0.jacobian(R3, S0.base_scalars())
G_metric = Jacob.T * Jacob
G_metric = G_metric.as_mutable().applyfunc(simplifications.get(CoordSystemType,simplify)).as_immutable()

if G_metric.is_diagonal():
  diagonal = [G_metric[i,i] for i in nrange]
  G_inv = diag(*[1/d for d in diagonal])
  G_sqrt = sqrt(reduce(mul,diagonal))
  G_sqrt = powers.get(CoordSystemType,simplify)(G_sqrt)
else:
  G_inv = G_metric.inv()
  G_sqrt = Jacob.det()

metric_Tensor = sum([ G_metric[i,j] * TensorProduct(S0.base_oneform(i), S0.base_oneform(j))
                      for (i,j) in zip(nrange,nrange) ])
                    # for i in range(S0.dim) for j in range(S0.dim) ])

displayLatex(metric_Tensor, intro = 'ds^2 =')
displayLatex(G_metric, intro = '(g_{ij}) = ')
displayLatex(G_sqrt, intro = r'\det (g_{ij}) = ')
# display(Markdown(rf'$\text{{Metric tensor}} {latex(G_metric)} \text{{with its determinant }} {latex(G_sqrt)}.$'))

$\displaystyle ds^2 = \sin^{2}{\left(\mathbf{\theta} \right)} \mathbf{r}^{2} \operatorname{d}\phi \otimes \operatorname{d}\phi + \mathbf{r}^{2} \operatorname{d}\theta \otimes \operatorname{d}\theta + \operatorname{d}r \otimes \operatorname{d}r$

$\displaystyle (g_{ij}) =  \left[\begin{matrix}1 & 0 & 0\\0 & \mathbf{r}^{2} & 0\\0 & 0 & \sin^{2}{\left(\mathbf{\theta} \right)} \mathbf{r}^{2}\end{matrix}\right]$

$\displaystyle \det (g_{ij}) =  \sin{\left(\mathbf{\theta} \right)} \mathbf{r}^{2}$

In [ ]:
from sympy.diffgeom import Differential
fr, ftheta, fphi = S0.base_scalars()
e_r, e_theta, e_phi = S0.base_vectors()
potential = 1/fr
Differential(potential)(e_r)

-1/r**2

In [ ]:
Differential(G_metric[1,1])(e_r)

2*(sin(theta)**2*sin(phi)**2*r + sin(theta)**2*cos(phi)**2*r + cos(theta)**2*r)*r/sqrt(sin(theta)**2*sin(phi)**2*r**2 + sin(theta)**2*cos(phi)**2*r**2 + cos(theta)**2*r**2)

##Gradient & Laplacian

Next, we define a function that returns the **gradient vector field** of a scalar field *s_field*. It first computes the contravariant components *tmp*, then the vector field is easily obtained by list comprehension.

**Note:** The CoordSystem *sys* and the inverse metric tensor *g_inv* have to be provided by the caller!

In [21]:
from sympy.diffgeom import Differential
func = Function('f')(*S0.base_scalars())

def grad_of_field(s_field, sys, g_inv):
  tmp = g_inv * Matrix([(Differential(s_field)(e)) for e in sys.base_vectors()])
  return sum([tmp[i]*sys.base_vector(i) for i in range(sys.dim)])

gradient = grad_of_field(func, S0, G_inv)
displayLeibniz(gradient, func, S0)
gradient

$\displaystyle  \frac{\partial f}{\partial \mathbf{r}}\partial_{r}+\frac{1}{\mathbf{r}^{2}}\frac{\partial f}{\partial \mathbf{\theta}}\partial_{\theta}+\frac{1}{\sin^{2}{\left(\mathbf{\theta} \right)} \mathbf{r}^{2}}\frac{\partial f}{\partial \mathbf{\phi}}\partial_{\phi}$

Subs(Derivative(f(_xi, theta, phi), _xi), _xi, r)*e_r + Subs(Derivative(f(r, _xi, phi), _xi), _xi, theta)*e_theta/r**2 + Subs(Derivative(f(r, theta, _xi), _xi), _xi, phi)*e_phi/(sin(theta)**2*r**2)

In [22]:
def div_of_field(v_field, sys, g_sqrt):
  tmp = g_sqrt * Matrix([dx(v_field) for dx in sys.base_oneforms()])
  return sum([simplifications.get(CoordSystemType,simplify)(Differential(tmp[i])(sys.base_vector(i))/g_sqrt)
              for i in range(sys.dim)])

laplace = powers.get(CoordSystemType,simplify)(div_of_field(gradient, S0, G_sqrt)) # .expand().trigsimp()
displayLeibniz(laplace,func,S0)
laplace

$\displaystyle \frac{1}{\mathbf{r}^{2}}\frac{\partial^{2} f}{\partial \mathbf{\theta}^{2}}+\frac{2 }{\mathbf{r}}\frac{\partial f}{\partial \mathbf{r}}+\frac{1}{\sin^{2}{\left(\mathbf{\theta} \right)} \mathbf{r}^{2}}\frac{\partial^{2} f}{\partial \mathbf{\phi}^{2}}+\frac{\cos{\left(\mathbf{\theta} \right)} }{\sin{\left(\mathbf{\theta} \right)} \mathbf{r}^{2}}\frac{\partial f}{\partial \mathbf{\theta}}+\frac{\partial^{2} f}{\partial \mathbf{r}^{2}}$

Subs(Derivative(f(_xi, theta, phi), (_xi, 2)), _xi, r) + 2*Subs(Derivative(f(_xi, theta, phi), _xi), _xi, r)/r + Subs(Derivative(f(r, _xi, phi), (_xi, 2)), _xi, theta)/r**2 + cos(theta)*Subs(Derivative(f(r, _xi, phi), _xi), _xi, theta)/(sin(theta)*r**2) + Subs(Derivative(f(r, theta, _xi), (_xi, 2)), _xi, phi)/(sin(theta)**2*r**2)

##Curvature

In [23]:
from sympy.diffgeom import metric_to_Christoffel_2nd
metric_to_Christoffel_2nd(metric_Tensor)

ValueError: The input expression concerns more than one coordinate systems, hence there is no unambiguous way to choose a coordinate system for the matrix.